In [39]:
from collections import defaultdict
from typing import Any

txt = "Hello World ! How are you ? World !"

In [50]:
default_int_dict: defaultdict[Any, int] = defaultdict(int)
# noinspection PyStatementEffect
default_int_dict["default"]
dict(default_int_dict)

{'default': 0}

In [54]:
default_none_dict: defaultdict[Any, None] = defaultdict(lambda: None)
# noinspection PyStatementEffect
default_none_dict["default"]
dict(default_none_dict)

{'default': None}

In [56]:
# noinspection PyTypeChecker
default_dict: defaultdict[Any, None] = defaultdict(lambda: [], {0:default_int_dict, 1:default_none_dict})
# noinspection PyStatementEffect
default_dict["default"]
dict(default_dict)

{0: defaultdict(int, {'default': 0}),
 1: defaultdict(<function __main__.<lambda>()>, {'default': None}),
 'default': []}

In [16]:
import itertools
list(itertools.combinations(range(7), 2))

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (3, 4),
 (3, 5),
 (3, 6),
 (4, 5),
 (4, 6),
 (5, 6)]

In [ ]:
Tree = NamedTuple("Tree", [("data", TreeType), ("children", list["Tree"])])

In [ ]:
import sys
from collections import defaultdict, OrderedDict
from time import perf_counter
from typing import Optional, NamedTuple

UP = "UP"
LEFT = "LEFT"
DOWN = "DOWN"
RIGHT = "RIGHT"
ORIENTATIONS = [UP, DOWN, LEFT, RIGHT]
START = perf_counter()


def auto_repr(cls):
    def __repr__(self):
        return '%s(%s)' % (type(self).__name__, ', '.join('%s=%s' % item for item in vars(self).items()))

    cls.__repr__ = __repr__
    return cls


def p(*args) -> None:
    [print(arg, file=sys.stderr, flush=True, end=" ") for arg in args]
    print(file=sys.stderr, flush=True)


def elapsed_time(debug=False) -> float:
    elapsed = perf_counter() - START
    if debug:
        p("{:.3f}".format(elapsed))
    return elapsed


class Coord(NamedTuple):
    x: int
    y: int

    def __str__(self):
        return "({}, {})".format(self.x, self.y)

    def __repr__(self):
        return "{} {}".format(self.x, self.y)


class World:
    def __init__(self):
        self.width: int = int(input())
        self.height: int = int(input())
        p("self.width", self.width)
        p("self.height", self.height)
        self.lines: list[str] = []
        self.cells: dict[Coord, Optional[Cell]] = defaultdict(lambda: None)
        self.is_optimized = False
        for y in range(self.height):
            self.lines.append(input())
            p(self.lines[-1])
            for x in range(self.width):
                self.cells[Coord(x, y)] = Cell(x, y, self.lines[y][x], self.cells)
        self.nodes: dict[Coord, Cell] = {coord: cell for (coord, cell) in self.cells.items() if cell and cell.is_node}
        # p("nodes", self.nodes)
        have_to_optimize = self.get_most_link_aimed()
        self.is_optimized = self.optimized_link(have_to_optimize)

    def get_most_link_aimed(self) -> list["Cell"]:
        return sorted([node for node in self.nodes.values() if not node.is_optimized()],
                      key=lambda x: x.link_left, reverse=True)

    def optimized_link(self, have_to_optimize, same_list=0) -> bool:
        # p("same check", repr(node))
        # p("reset", k, "links_towards", repr(attainable_node))
        # p("after reset", have_to_optimize)
        # p("have_to_optimize", have_to_optimize)
        p(same_list)
        if len(have_to_optimize) == 0:
            return True
        elif elapsed_time() >= 0.5:
            output(self)
            p(elapsed_time())
            return False
        elif same_list == 1:
            node = have_to_optimize[0]
            for k, attainable_coord in node.attainable_coord.items():
                if attainable_coord:
                    attainable_node = self.nodes[attainable_coord]
                    for coord_linked in attainable_node.links_towards:
                        attainable_node.link_left += 1
                        self.nodes[coord_linked].link_left += 1
                    attainable_node.links_towards = []
        elif same_list == 0:
            for node in have_to_optimize:
                node.add_attainable_links(self.nodes)
        elif same_list == 2:
            for node in have_to_optimize:
                node.add_attainable_links(self.nodes)
        save_list = have_to_optimize
        have_to_optimize = self.get_most_link_aimed()
        if save_list == have_to_optimize:
            return self.optimized_link(have_to_optimize, same_list + 1)
        return self.optimized_link(have_to_optimize)


@auto_repr
class Cell:
    def __init__(self, x: int, y: int, descr: str, cells: dict[Coord, Optional["Cell"]]):
        self.x: int = x
        self.y: int = y
        self.coord: Coord = Coord(x, y)
        self.attainable_coord: OrderedDict[str, Coord] = OrderedDict()
        self.is_node: bool = descr != "."
        self.link_left: int = 0
        self.links_towards: list["Coord"] = []
        if self.is_node:
            self.link_left = int(descr)
            for orientation in ORIENTATIONS:
                attainable_coord = self.get_closest_coord(cells, orientation)
                if attainable_coord:
                    self.attainable_coord[orientation] = attainable_coord
                    attainable_node = cells[attainable_coord]
                    pos = ORIENTATIONS.index(orientation)
                    opposite_orientation = ORIENTATIONS[(pos + (-1 if pos % 2 else 1)) % len(ORIENTATIONS)]
                    attainable_node.attainable_coord[opposite_orientation] = self.coord
                    self.add_attainable_links(cells)

    def __str__(self):
        return "({} {})".format(self.coord, self.attainable_coord)

    def get_closest_coord(self, cells: dict[Coord, Optional["Cell"]], orientation: str) -> Optional["Coord"]:
        x, y = self.x, self.y
        while True:
            x -= orientation == LEFT
            y -= orientation == UP
            x += orientation == RIGHT
            y += orientation == DOWN
            cursor = cells[Coord(x, y)]
            if cursor is None:
                return None
            elif cursor.is_node:
                return cursor.coord

    def is_optimized(self) -> bool:
        return self.link_left == 0

    def add_link(self, node: "Cell") -> bool:
        if node.coord not in self.links_towards and not self.is_optimized() and not node.is_optimized():
            self.links_towards.append(node.coord)
            self.link_left -= 1
            node.link_left -= 1
            if not self.is_optimized() and not node.is_optimized() and self.coord not in node.links_towards:
                # p("miror")
                return node.add_link(self)
            return True
        return False

    def sort_attainable_coord(self, nodes):
        for key, _ in sorted(self.attainable_coord.items(), key=lambda x: nodes[x[1]].link_left if x[1] else 0):
            self.attainable_coord.move_to_end(key)

    def add_attainable_links(self, cells):
        self.sort_attainable_coord(cells)
        for attainable_coord in self.attainable_coord.values():
            if attainable_coord and attainable_coord not in self.links_towards:
                attainable_node = cells[attainable_coord]
                self.add_link(attainable_node)


def output(world) -> None:
    for node in world.nodes.values():
        for link in node.links_towards:
            print(repr(node.coord), repr(link), 1)


if __name__ == '__main__':
    timer = perf_counter()
    p(sys.version_info)
    world = World()
    if world.is_optimized:
        output(world)
    elapsed_time()
